Henry Pacheco Cachon

Created: 1/30/2021 Last Modified: 02/02/2021

The purpose of this notebook is to create a fourlevel array which can be used in c++ in order to run computations quickly. This is more to see if it's or not!

In [17]:
import numpy as np
from sympy import *
import nStates as ns 
import matplotlib.pyplot as plt 
import pandas as pd
from scipy import sparse
from numba import jit

In [18]:
# Initializing all variables

kB = 1.381e-23 
AMU = 1.66e-27 
mAtom = 39
temperature = 323
cellLength = 7.0
u = np.sqrt(2*kB*temperature/(mAtom*AMU))*100
logP = 9.967 - 4646/temperature
pressure = 10 ** logP
density = (pressure)/(kB*temperature) * 1e-06

gamma1 = 6.0
gamma2 = 1.0
gamma3 = 0.1

gammaP = 0.2
gammaC = 0.2

omegaP = 2.5
omegaC12 = 2.0
omegaC13 = 0.9

deltaP = 0.0
delta23 = 5.0

lambdaP = 767 * 1e-07
lambdaC = 696 * 1e-07

alphaP_0 = (3/(2*np.pi)) * lambdaP**2 * density
alphaC_0 = (3/(2*np.pi)) * lambdaC**2 * density

k_P = 1/lambdaP * 1e-06
k_C = 1/lambdaC * 1e-06

couplingDetunings = np.linspace(-60,40,201)
velocities = np.linspace(-10000,10000,201)
dv = velocities[1] - velocities[0]

In [19]:
# Initializing the symbols for our hamiltonian
dp0, rabi01, rabi12, rabi13, dc0, Delta23, kp, kc, v = symbols('delta_p Omega_01 Omega_12 Omega_13 delta_c Delta_23 k_p k_c v')

# Initializing our hamiltonian array
H = np.array([[-(dp0+kp*v), 1/2*rabi01, 0, 0],[1/2*rabi01, 0, 1/2*rabi12, 1/2*rabi13],[0, 1/2*rabi12, (dc0-kc*v), 0],[0, 1/2*rabi13, 0, (dc0-kc*v) + Delta23]])

# Initializing the allowed transitions in our system
transitions = [(1,0),(2,1),(3,1)]

# Dictionary of parameters that we want to hold constant
static = {Symbol('rho_00') : 1-Symbol('rho_11')-Symbol('rho_22')-Symbol('rho_33')}

# Initializing all of our static parameters and hamiltonian into the States class 
fourLevel = ns.States(numberStates=4, hamiltonian=H, staticDictionary=static)

In [20]:
constants = [dp0, rabi01, rabi12, rabi13, dc0, Delta23, kp, kc, v, Symbol("Gamma_21"), Symbol("Gamma_31"), Symbol('Gamma_10'), Symbol('gamma_p'), Symbol('gamma_c')]

In [21]:
# Creating our lindblad matrix with allowed transitions
fourLevel.lindbladMatrix(transitions,laser=True)

# Creating our density matrix
fourLevel.densityMatrix()

# Symbolically computing the i[p,H] + L matrix
fourLevel.superMatrix()

# Extracting the density matrix equations from the super matrix
fourLevel.makeEquations()

symbolicMatrix = fourLevel.changeParam(constants,cpp=True)

I want the output to be this matrix 

$$ 

\left[\begin{array}{ccccccccccccccc}-6.0 & 1.0 & 0.1 & - 1.25 i & 0 & 0 & 1.25 i & 1.0 i & 0.45 i & 0 & - 1.0 i & 0 & 0 & - 0.45 i & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & - 1.0 i & 0 & 0 & 1.0 i & 0 & 0 & 0 & 0\\0 & 0 & -0.1 & 0 & 0 & 0 & 0 & 0 & - 0.45 i & 0 & 0 & 0 & 0 & 0.45 i & 0\\- 2.5 i & - 1.25 i & - 1.25 i & 1.0 i k_{p} v - 2.8 & 1.0 i & 0.45 i & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 i & 1.0 i \left(\delta_{c} - k_{c} v + k_{p} v\right) - 0.9 & 0 & 0 & - 1.25 i & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0.45 i & 0 & 1.0 i \left(\Delta_{23} + \delta_{c} - k_{c} v + k_{p} v\right) - 0.45 & 0 & 0 & - 1.25 i & 0 & 0 & 0 & 0 & 0 & 0\\2.5 i & 1.25 i & 1.25 i & 0 & 0 & 0 & - 1.0 i k_{p} v - 3.2 & 0 & 0 & - 1.0 i & 0 & 0 & - 0.45 i & 0 & 0\\1.0 i & - 1.0 i & 0 & 0 & - 1.25 i & 0 & 0 & 1.0 i \left(\delta_{c} - k_{c} v\right) - 3.7 & 0 & 0 & 0 & 0 & 0 & 0 & - 0.45 i\\0.45 i & 0 & - 0.45 i & 0 & 0 & - 1.25 i & 0 & 0 & 1.0 i \left(\Delta_{23} + \delta_{c} - k_{c} v\right) - 3.25 & 0 & 0 & - 1.0 i & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & - 1.0 i & 0 & 0 & 1.0 i \left(- k_{p} v - \left(\delta_{c} - k_{c} v\right)\right) - 0.9 & 1.25 i & 0 & 0 & 0 & 0\\- 1.0 i & 1.0 i & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.25 i & - 1.0 i \left(\delta_{c} - k_{c} v\right) - 3.7 & 0.45 i & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & - 1.0 i & 0 & 0.45 i & 1.0 i \left(\Delta_{23} + \delta_{c} - k_{c} v - \left(\delta_{c} - k_{c} v\right)\right) - 0.55 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & - 0.45 i & 0 & 0 & 0 & 0 & 0 & 1.0 i \left(- k_{p} v - \left(\Delta_{23} + \delta_{c} - k_{c} v\right)\right) - 0.45 & 1.25 i & 0\\- 0.45 i & 0 & 0.45 i & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.25 i & - 1.0 i \left(\Delta_{23} + \delta_{c} - k_{c} v\right) - 3.25 & 1.0 i\\0 & 0 & 0 & 0 & 0 & 0 & 0 & - 0.45 i & 0 & 0 & 0 & 0 & 0 & 1.0 i & 1.0 i \left(\delta_{c} - k_{c} v - \left(\Delta_{23} + \delta_{c} - k_{c} v\right)\right) - 0.55\end{array}\right]


$$ 

This is a huge matrix and I really don't want to be the one making this by hand in C++ 

In [22]:
from sympy.printing.c import C11CodePrinter

In [23]:
printer = C11CodePrinter()

In [24]:
A = MatrixSymbol('A',15,15)

In [25]:
test = printer.doprint(symbolicMatrix,assign_to=A)

In [26]:
Lines = test.split('\n')

In [27]:
from re import search

In [28]:
row = 0
matrixAssign = ".coeffRef({},{})"
complexFormat = "dcomplex({},{});"
newLines = []
for string in Lines:

    #Getting Indexes
    l = string.index('[')
    r = string.index(']')
    number = int(string[l+1:r])

    x = number%15
    y = row

    #Get values
    eq = string.index('=')
    stop = string.index(';')
    value = string[eq+2:stop]
    
    if value != '0':

        if search('I',value):

            if value.rfind(' ') == -1:
                pro = value.index('*')
                newComplex = complexFormat.format(0,value.replace('I*',''))
            else:
                bigBoi = value.split(' ')
                a = ''
                b = ''
                check = 0

                for i in range(len(bigBoi)):
                    val = bigBoi[i]
                    if val.find("I") != -1:
                        check = i
                
                bigBoi[check:] = [''.join(bigBoi[check:])]

                for i in range(len(bigBoi)):
                    val = bigBoi[i]
                    if val.find("I") != -1:
                        sign = bigBoi[i-1]
                        if sign == "+":
                            b += val
                        else:
                            b = sign + val
                    else:
                        a += val
                
                a = a[:-1]
                b = b.replace("I*",'')
                newComplex = complexFormat.format(a,b)
            
        else:
            newComplex = complexFormat.format(value,0)

        newString = string[:l] + matrixAssign.format(y,x) + string[r+1:r+4] + newComplex +'\n'
        newLines.append(newString)

    if x == 14:
        row +=1
    

In [29]:
newVar = "double {};\n"
globalVars = [newVar.format(i) for i in constants] + ['\n','\n']
function = ["SparseMatrix<dcomplex> M(delta_c, v){\n","\n"]

In [30]:
allLines = globalVars + function + ["   "+ string for string in newLines] + ['\n','   A.makeCompressed();\n','\n','   return A;\n \n','}']

In [31]:
testFile = open("test.txt",'w')
testFile.writelines(allLines)
testFile.close()